In [1]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from IPython.display import clear_output
from tkinter import Tk, filedialog

from quick_pp.objects import Well, Project
import quick_pp.las_handler as las

In [3]:
root = Tk()
file_objects = filedialog.askopenfiles(title='Choose well Log ASCII Standard (LAS) files to be combined',
                                        filetype=(('LAS Files', '*.LAS *.las'), ('All Files', '*.*')),
                                        multiple=True,
                                        mode='rb')
root.destroy()
if file_objects:
    project_name = "KSW"    
    project = Project(name=project_name)
    project.read_las([f.name for f in file_objects])
    project.save()

clear_output()

In [ ]:
# TODO: Rename and filter columns
import pandas as pd

df = pd.read_parquet(r"C:\projects\ERMAI\data\03_primary\BEKK_CLEAN_1b8c72.parquet")
# df.sort_values(['WELL_NAME', 'DEPTH'], inplace=True)

In [ ]:
marker_df = pd.read_csv(r"C:\projects\quick_pp\notebooks\data\04_project\BEKK_MARKER.csv")

marker_df['WELL_NAME'] = marker_df['Well identifier (UWI)']
marker_df['DEPTH'] = marker_df['MDDF(ft)']
marker_df['ZONES'] = marker_df['Surface']
# marker_df.sort_values(['WELL_NAME', 'DEPTH'], inplace=True)

In [ ]:
marker_df[marker_df.WELL_NAME == 'BEKK-005'].tail(20)

In [ ]:
return_df = pd.DataFrame()
for well, data in df.drop('ZONES', axis=1).groupby('WELL_NAME'):
    temp_df = marker_df[marker_df.WELL_NAME == well][['DEPTH', 'ZONES']].copy().sort_values('DEPTH')
    temp_df = pd.merge_asof(
        data, temp_df, on='DEPTH', direction='forward', tolerance=50
    )
    return_df = pd.concat([return_df, temp_df])


In [ ]:
project_name = "BEKK_CLEAN"    
project = Project(name=project_name)
project.update_data(return_df)
project.save()

In [ ]:
# Loading a dataframe

root = Tk()
file_objects = filedialog.askopenfiles(title='Choose well Log ASCII Standard (LAS) files to be combined',
                                        filetype=(('LAS Files', '*.LAS *.las'), ('All Files', '*.*')),
                                        multiple=True,
                                        mode='rb')
root.destroy()
df, _ = las.read_las_files(file_objects)